In [1]:
import os
import shutil
import logging
from pathlib import Path

# Configure logging for debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define paths
NOTEBOOK_DIR = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
MODELS_BASE_PATH = os.path.join('data', 'Models')  # Adjust if your models are elsewhere
SHOW_RESULTS_PATH = os.path.join(NOTEBOOK_DIR, 'show_results', 'visualization')

# Model configuration (from your previous context)
MODEL_CONFIG = {
    'no_mask': {
        'name': 'voxelmorph_no_mask',
        'use_mask': False,
        'use_mse_mask': False,
        'use_smoothness_mask': False
    },
    # (Other configurations are commented out in your setup)
}

LAMBDAS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

KERNEL_CONFIGS = {
    'default': {
        'encoder': [[3], [3], [3], [3]],
        'decoder': [[3], [3], [3], [3]],
        'final': [3, 3, 3]
    },
    'first5': {
        'encoder': [[5], [3], [3], [3]],
        'decoder': [[3], [3], [3], [3]],
        'final': [3, 3, 3]
    },
    'first7_second5': {
        'encoder': [[7], [5], [3], [3]],
        'decoder': [[3], [3], [3], [3]],
        'final': [3, 3, 3]
    }
}

KERNEL_KEYS = list(KERNEL_CONFIGS.keys())

# Sample names
SAMPLE_NAMES = [f'sample_{i}_batch_0.png' for i in range(4)]  # ['sample_0_batch_0.png', ..., 'sample_3_batch_0.png']

def organize_sample_images():
    """
    Organizes sample images from model results into a new directory structure for easy comparison.
    Copies sample_X_batch_0.png from each model's results/debug_analysis into
    show_results/visualization/sample_X_batch_0/, renaming each image to the model name.
    """
    # Create the destination base directory
    os.makedirs(SHOW_RESULTS_PATH, exist_ok=True)
    logging.info(f"Created base directory: {SHOW_RESULTS_PATH}")

    # Create a folder for each sample
    sample_dirs = {}
    for sample_name in SAMPLE_NAMES:
        sample_dir = os.path.join(SHOW_RESULTS_PATH, sample_name.replace('.png', ''))
        os.makedirs(sample_dir, exist_ok=True)
        sample_dirs[sample_name] = sample_dir
        logging.info(f"Created sample directory: {sample_dir}")

    # Iterate through each model configuration to find and copy sample images
    total_copied = 0
    for model_key in MODEL_CONFIG:
        for kernel_key in KERNEL_KEYS:
            for lambda_val in LAMBDAS:
                # Model folder name (using :.3f as per your actual directory naming)
                model_name = f"{MODEL_CONFIG[model_key]['name']}_kernel_{kernel_key}_lambda_{lambda_val:.3f}"
                model_dir = os.path.join(MODELS_BASE_PATH, model_name)
                debug_analysis_dir = os.path.join(model_dir, 'results', 'debug_analysis')

                # Check if debug_analysis directory exists
                if not os.path.exists(debug_analysis_dir):
                    logging.warning(f"Debug analysis directory not found: {debug_analysis_dir}")
                    continue

                # Look for each sample image in the debug_analysis directory
                for sample_name in SAMPLE_NAMES:
                    src_path = os.path.join(debug_analysis_dir, sample_name)
                    if not os.path.exists(src_path):
                        logging.warning(f"Sample image not found: {src_path}")
                        continue

                    # Destination path: show_results/visualization/sample_X_batch_0/<model_name>.png
                    dest_dir = sample_dirs[sample_name]
                    dest_path = os.path.join(dest_dir, f"{model_name}.png")

                    # Copy the image
                    try:
                        shutil.copy2(src_path, dest_path)
                        total_copied += 1
                        logging.info(f"Copied {src_path} to {dest_path}")
                    except Exception as e:
                        logging.error(f"Failed to copy {src_path} to {dest_path}: {str(e)}")

    logging.info(f"Total images copied: {total_copied}")

if __name__ == "__main__":
    organize_sample_images()

2025-06-04 20:14:28,298 - INFO - Created base directory: d:\study\graduation_project\VXM\show_results\visualization
2025-06-04 20:14:28,300 - INFO - Created sample directory: d:\study\graduation_project\VXM\show_results\visualization\sample_0_batch_0
2025-06-04 20:14:28,302 - INFO - Created sample directory: d:\study\graduation_project\VXM\show_results\visualization\sample_1_batch_0
2025-06-04 20:14:28,305 - INFO - Created sample directory: d:\study\graduation_project\VXM\show_results\visualization\sample_2_batch_0
2025-06-04 20:14:28,417 - INFO - Created sample directory: d:\study\graduation_project\VXM\show_results\visualization\sample_3_batch_0
2025-06-04 20:14:28,718 - INFO - Copied data\Models\voxelmorph_no_mask_kernel_default_lambda_0.100\results\debug_analysis\sample_0_batch_0.png to d:\study\graduation_project\VXM\show_results\visualization\sample_0_batch_0\voxelmorph_no_mask_kernel_default_lambda_0.100.png
2025-06-04 20:14:28,721 - INFO - Copied data\Models\voxelmorph_no_mask_